In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, TrainingArguments, Trainer
import spacy
import torch
from spacy.language import Language
from spacy import displacy
import time
import glob
import re
import math
import statistics
import os
import json
import calendar
import holidays
from pathlib import Path
from datetime import date
from datetime import datetime
import pandas as pd
import numpy as np
import collections
import hashlib
from dateutil.parser import parse
import shutil
import ast
from io import StringIO
import requests
import glob

In [2]:
import spacy
from spacy.language import Language
from spacy import displacy
import time

@Language.component("newsent")
def set_custom_boundaries(doc):
    for token in doc[:-1]:
        #print(token.text, token.text in ("’s", "'s"))
        if token.text.upper() in (";", "--", "\n\n", "\n", "QUARTERLY", "STORY", "\n\n\n\n", "\n\n\n"):
            #print("Detected:", token.text)
            doc[token.i].is_sent_start = True
    return doc

#spacy.require_gpu()
nlp = spacy.load("../../Summary/NER/RelateEntity/train/model-best-local")
nlp.add_pipe('sentencizer')
nlp.add_pipe('newsent', name="newsent", last=True)

c:\users\ankan\appdata\local\programs\python\python37\lib\site-packages\spacy\util.py:837: UserWarning: [W095] Model 'en_pipeline' (0.0.0) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.3.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
c:\users\ankan\appdata\local\programs\python\python37\lib\site-packages\spacy_transformers\pipeline_component.py:406: UserWarning: Automatically converting a transformer component from spacy-transformers v1.0 to v1.1+. If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spacy-transformers version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


<function __main__.set_custom_boundaries(doc)>

In [3]:
def isScanningRqd(pfile):
    with open(pfile, 'r', encoding = "utf-8") as fp:
        for l_no, line in enumerate(fp):
            if "NOPAD***" in line:
                return False
    return True

In [4]:
def getSentences(inputfile, nlp, text=None):
    if(not text):
        with open(inputfile, 'r', encoding="utf-8") as f:
            text = f.read()

    doc = nlp(text)
    sentences = [str(sent).strip() for sent in doc.sents]

    #print(len(sentences))
    return(sentences)

In [5]:
rplStr = ["PG*** ", "ED*** ", "SCHQ*** ", "SCBQ*** ", "SCBF*** ", "SCHF*** ", "SCG*** ", "GF*** ", "GQ*** ", "SC*** ", "NOPAD*** "]
stag = ["SCHQ***", "SCHF***", "SCBQ***", "SCBF***", "SCG***"]
gtag = ["GF***", "GQ***"]
ptag = "PG***"

def preProcessSent(line):
    for s in rplStr:
        line = line.replace(s, "")
    if("TBLST***" in line or "TBLET***" in line or "CS***" in line or line == "\n" or line == "\n\n" or "https://finance.yahoo.com" in line):
        return(line, False)
    line = line.replace("\n", "")
    if(line == None or line == ""):
        return(line, False)
    return(line, True)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")

def query1_from_list(context):
    ans = "{\"RELATIONS\": [\"KEY:GAAP Net Income Per Share!!TYPE:OUT!!MONEY:$0.38!!LINK:KV\"]}"
    sent = ["GAAP Net Income Per Share is $0.38 in second quarter 2023 @@@"]
    
    ans1 = "{\"RELATIONS\": [\"KEY:Non-GAAP net income!!TYPE:OUT!!MONEY:$110.1 MN!!LINK:KV\"]}"
    sent1 = ["Non-GAAP net income is $110.1 million in third quarter 2022 @@@"]
    
    ans2 = "{\"RELATIONS\": [\"KEY:Non-GAAP Net Income Per Share!!TYPE:OUT!!MONEY:-$0.13!!LINK:KV\"]}"
    sent2 = ["Non-GAAP Net Income Per Share is ($0.13) in first quarter 2023 @@@"]
    
    tfewshot = f"""
    Question: What are the relations present in the following text? 
    
    Context: {" * ".join(sent)}. 
    
    Answer: {ans}.
    
    """
    tfewshot += f"""
    Question: What are the relations present in the following text? 
     
    Context: {" * ".join(sent1)}. 
    
    Answer: {ans1}.
    
    """
    tfewshot += f"""
    Question: What are the relations present in the following text?
    
    Context: {" * ".join(sent2)}. 
    
    Answer: {ans2}
    
    """
    #print(tfewshot)
    #print("\n\n")
    t5query = f"""{tfewshot}
    Question: What are the relations present in the following text? 
    
    Context:  {" * ".join(context)}.
    
    Answer:
    
    """
    print(t5query)
    inputs = tokenizer(t5query, return_tensors="pt")
    outputs = model.generate(**inputs, max_new_tokens=100)
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

context = ["GAAP Gross profit for the third quarter of 2022 was $210 million"]
result = query1_from_list(context)
print(f"{result[0]}")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base", torch_dtype=torch.bfloat16)

def query1_from_list(context):
    ans = "{\"RELATIONS\": [\"KEY:GAAP Net Income Per Share!!TYPE:OUT!!MONEY:$0.38!!LINK:KV\"]}"
    sent = ["GAAP Net Income Per Share is $0.38 in second quarter 2023 @@@"]
    
    ans1 = "{\"RELATIONS\": [\"KEY:Non-GAAP net income!!TYPE:OUT!!MONEY:$110.1 MN!!LINK:KV\"]}"
    sent1 = ["Non-GAAP net income is $110.1 million in third quarter 2022 @@@"]
    
    ans2 = "{\"RELATIONS\": [\"KEY:Non-GAAP Net Income Per Share!!TYPE:OUT!!MONEY:-$0.13!!LINK:KV\"]}"
    sent2 = ["Non-GAAP Net Income Per Share is ($0.13) in first quarter 2023 @@@"]
    
    tfewshot = f"""
    Article: {" * ".join(sent)}
    
    Question: What are the relations present in the text? Display it in json format. {ans}
    
    """
    tfewshot += f"""
    Article: {" * ".join(sent1)}. 
    
    Question: What are the relations present in the text? Display it in json format. {ans1}
    
    """
    tfewshot += f"""
    Article: {" * ".join(sent2)}.
    
    Question: What are the relations present in the text? Display it in json format. {ans2}
    
    """
    #print(tfewshot)
    #print("\n\n")
    t5query = f"""{tfewshot}
    Article: {" * ".join(context)}.
    
    Question: What are the relations present in the text? Display it in json format."""
    print(t5query)
    inputs = tokenizer(t5query, return_tensors="pt")
    outputs = model.generate(**inputs, max_new_tokens=100)
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

context = ["GAAP Gross profit for the third quarter of 2022 was $210 million"]
result = query1_from_list(context)
print(f"{result[0]}")

In [ ]:
# Format 2, includiing entities and relations
maxFiles = 60
fileCnt = 0
source = "../../Summary/DATA/FLAN/Backup/Format-1"
files = glob.glob(source+"/*_ER.tsv")
#print(files)
for file in files:
    #print(file)
    basefile = os.path.basename(file)
    #print(basefile)
    outfile = "../../Summary/DATA/FLAN/Train/"+basefile
    of = None
    #print(outfile)
    if outfile:
        outfilePath = Path(outfile)
        if outfilePath.is_file():
            print("Output File {} already exists".format(outfile))
            continue
        
        if(maxFiles > 0):
            fileCnt = fileCnt + 1
            if(fileCnt > maxFiles):
                break
            
        print("Creating Output File {}".format(outfile))
        of = open(outfilePath, "w", encoding = "utf-8")
        
        
    with open(file, "r", encoding = "ISO-8859-1") as f:
        line = f.readline()
        #print(line)
        while line:
            if ("RELATIONS" not in line):
                of.write(line)
                #of.write("\n")
            else:
                relations = dict()
                nsplit = line.split("\t")
                relation = nsplit[2]
                relation = relation.replace("\n","")
                sent = nsplit[1]
                sent = sent.replace("\t","")
                #print(sent)
                relation = json.loads(relation)
                #print(relation)
                sentences = getSentences(None, nlp, sent)
                nerl = None
                for l in sentences:
                    text1 = list()
                    text1.append(l)
                    for doc in nlp.pipe(text1, disable=["tagger"]):
                        for ent in doc.ents:
                            if(not nerl):
                                nerl = ent.label_.replace(":","").replace(",","")+"="+ent.text.replace(":","").replace(",","")
                            else:
                                nerl = nerl + "," + ent.label_.replace(":","").replace(",","")+"="+ent.text.replace(":","").replace(",","")
                #relations["ENTITIES"] = None
                relations["ENTITIES"] = list()
                relations["ENTITIES"].append(nerl)
                #rlist = list()
                #for rstr in relation["RELATIONS"]:
                #    rstr = rstr.replace(",MONEY","!!MONEY").replace(",TYPE","!!TYPE").replace(",CD","!!CD").replace(",RELATION","!!RELATION").replace(",PCT","!!PCT").replace(",DATE","!!DATE").replace(",CALENDAR","!!CALENDAR")
                #    rlist.append(rstr)
                relations["RELATIONS"] = relation["RELATIONS"]
                #print(relations)
                nsplit[2] = json.dumps(relations)
                nsplit[1] = sent
                nline = "\t".join(nsplit)
                #nline = nline + "</s>" #Not Required
                of.write(nline)
                of.write("\n")
            line = f.readline()
    if(of):
        of.close()
    #break

In [ ]:
# Format 3, combining entities, relations and earning section in single training data
maxFiles = -1
fileCnt = 0
source = "../../Summary/DATA/FLAN/Backup/Format-2"

files = glob.glob(source+"/*_ER.tsv")
#print(files)
for file in files:
    #print(file)
    basefile = os.path.basename(file)
    #print(basefile)
    outfile = "../../Summary/DATA/FLAN/Train/"+basefile
    of = None
    #print(outfile)
    if outfile:
        outfilePath = Path(outfile)
        if outfilePath.is_file():
            print("Output File {} already exists".format(outfile))
            continue
        
        if(maxFiles > 0):
            fileCnt = fileCnt + 1
            if(fileCnt > maxFiles):
                break
            
        print("Creating Output File {}".format(outfile))
        of = open(outfilePath, "w", encoding = "utf-8")
        
    
    tag = None
    section = None
    ssection = None
    header = None
    report = "REGULAR"
    
    with open(file, "r", encoding = "ISO-8859-1") as f:
        line = f.readline()
        #print(line)
        while line:
            if ("RELATIONS" not in line):
                of.write(line)
                #of.write("\n")
            else:
                relations = dict()
                nsplit = line.split("\t")
                relation = nsplit[2]
                relation = relation.replace("\n","")
                sent = nsplit[1]
                sent = sent.replace("\t","")
                tag = sent.split(" ")[0].strip()
                #print(tag)
                #print(sent)
                if(tag in stag):
                    section = sent.replace(tag, "").strip()
                    header = section
                    if("Q" in tag):
                        report = "REGULAR"
                    elif("F" in tag):
                        report = "REGULARFULL"
                    elif("G" in tag):
                        report = "GUIDE"
                    #print(sent)
                elif(tag in gtag):
                    ssection = sent.replace(tag, "").strip()
                    header = ssection
                    if("Q" in tag):
                        report = "GUIDE"
                    elif("F" in tag):
                        report = "GUIDEFULL"
                    #if(header):
                    #    if("***" in tag):
                    #        sent = sent.replace(tag, "")
                    #        sent = tag + " " + header + " " + sent
                    #    else:
                    #        sent = header + " " + sent
                    #if(section):
                    #    header = section + " " + ssection
                    #else:
                    #    header = ssection
                    #print(sent)
                else:
                    if(header):
                        if("***" in tag):
                            sent = sent.replace(tag, "")
                            sent = tag + " " + header + " " + sent
                        else:
                            sent = header + " " + sent
                    #print(sent)
                relation = json.loads(relation)
                #print(relation)
                relations["ENTITIES"] = relation["ENTITIES"]
                rltn = relation["RELATIONS"]
                if(len(rltn) != 0 and rltn[0] != ''):
                    rlist = list()
                    for rstr in rltn:
                        rstr = rstr + "!!SECTION:"+ report
                        rlist.append(rstr)
                    relations["RELATIONS"] = rlist
                else:
                    relations["RELATIONS"] = rltn 
                #print(relations)
                nsplit[2] = json.dumps(relations)
                nsplit[1] = sent
                nline = "\t".join(nsplit)
                #nline = nline + "</s>" #Not Required
                of.write(nline)
                of.write("\n")
            line = f.readline()
    if(of):
        of.close()
    #break

In [6]:
devDataFile = "../../Summary/DATA/FLAN/Dev/dev.tsv"
trainDataFile = "../../Summary/DATA/FLAN/Train/train.tsv"
testDataFile = "../../Summary/DATA/FLAN/Test/test.tsv"

trainDir = "../../Summary/DATA/FLAN/Train"
devDir = "../../Summary/DATA/FLAN/Dev"
testDir = "../../Summary/DATA/FLAN/Test"

def writeTrainingData(writeFile, writeDir):
    files = glob.glob(writeDir+"/*_ER.tsv")
    print(files)
    frames = list()

    if(len(files) > 0):
        for file in files:
            print(file)
            #df = pd.read_csv(file, sep="\t", encoding = "utf-8").astype(str)
            df = pd.read_csv(file, sep="\t", encoding = "ISO-8859-1").astype(str)
            df = df.dropna()
            df = df[df['Sentence1'].notna()]
            #print(df)
            frames.append(df)
    result = pd.concat(frames)
    print(result)
    result.to_csv(writeFile, sep='\t', index=False, header=True)

In [ ]:
writeTrainingData(trainDataFile, trainDir)
writeTrainingData(devDataFile, devDir)
writeTrainingData(testDataFile, testDir)

In [ ]:
train_df = pd.read_csv(trainDataFile, sep="\t", encoding = "utf-8").astype(str)
eval_df = pd.read_csv(devDataFile, sep="\t", encoding = "utf-8").astype(str)
test_df = pd.read_csv(testDataFile, sep="\t", encoding = "utf-8").astype(str)

train_df = train_df.rename(
    columns={"Sentence1": "input_text", "Sentence2": "target_text"}
)
eval_df = eval_df.rename(
    columns={"Sentence1": "input_text", "Sentence2": "target_text"}
)
test_df = test_df.rename(
    columns={"Sentence1": "input_text", "Sentence2": "target_text"}
)

train_df = train_df[["input_text", "target_text"]]
eval_df = eval_df[["input_text", "target_text"]]
test_df = test_df[["input_text", "target_text"]]

#train_df["prefix"] = "paraphrase"
#train_df = train_df[["prefix", "input_text", "target_text"]]
#train_df = train_df[["input_text", "target_text"]]

#eval_df["prefix"] = "paraphrase"
#eval_df = eval_df[["prefix", "input_text", "target_text"]]
#eval_df = eval_df[["input_text", "target_text"]]

train_df = train_df.dropna()
train_df = train_df[train_df['input_text'].notna()]

eval_df = eval_df.dropna()
eval_df = eval_df[eval_df['input_text'].notna()]

test_df = test_df.dropna()
test_df = test_df[test_df['input_text'].notna()]

#train_df["input_text"] = train_df["input_text"].apply(clean_unnecessary_spaces)
#train_df["target_text"] = train_df["target_text"].apply(clean_unnecessary_spaces)
print("TRAIN DATA ..............")
print(train_df)

#eval_df["input_text"] = eval_df["input_text"].apply(clean_unnecessary_spaces)
#eval_df["target_text"] = eval_df["target_text"].apply(clean_unnecessary_spaces)
print("EVAL DATA ..............")
print(eval_df)

print("TEST DATA ..............")
print(test_df)

In [7]:
#modelPath = "./feroutputs/checkpoint-5203/pytorch_model.bin"
modelPath = "./feroutputs/pytorch_model.bin"
modelDir = "./feroutputs/"
model_def = 'google/flan-t5-small'
if os.path.isfile(modelPath):
    model_name = "./feroutputs/checkpoint-5203"
else:
    di = sorted(os.listdir(modelDir), reverse=True)
    if(len(di) > 0):
        model_name = modelDir+(di[0])
    else:
        model_name = model_def
print(model_name)

./feroutputs/checkpoint-5203


In [8]:

#original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
#tokenizer = AutoTokenizer.from_pretrained(model_name)
#model_name = "facebook/bart-base"

#original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
#original_model.to('cuda')

In [9]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(original_model))

trainable model parameters: 76961152
all model parameters: 76961152
percentage of trainable model parameters: 100.00%


In [ ]:
inputText = (train_df["input_text"][0])
outputText = (train_df["target_text"][0])

ans = "{\"RELATIONS\": [\"KEY:GAAP Net Income Per Share!!TYPE:OUT!!MONEY:$0.38!!LINK:KV\"]}"
sent = ["GAAP Net Income Per Share is $0.38 in second quarter 2023 @@@"]

ans1 = "{\"RELATIONS\": [\"KEY:Non-GAAP net income!!TYPE:OUT!!MONEY:$110.1 MN!!LINK:KV\"]}"
sent1 = ["Non-GAAP net income is $110.1 million in third quarter 2022 @@@"]

ans2 = "{\"RELATIONS\": [\"KEY:Non-GAAP Net Income Per Share!!TYPE:OUT!!MONEY:-$0.13!!LINK:KV\"]}"
sent2 = ["Non-GAAP Net Income Per Share is ($0.13) in first quarter 2023 @@@"]

tfewshot = f"""
Article: {" * ".join(sent)}

Question: What are the relations present in the text? Display it in json format. {ans}

"""
tfewshot += f"""
Article: {" * ".join(sent1)}. 

Question: What are the relations present in the text? Display it in json format. {ans1}

"""
tfewshot += f"""
Article: {" * ".join(sent2)}.

Question: What are the relations present in the text? Display it in json format. {ans2}

"""
#print(tfewshot)
#print("\n\n")
t5query = f"""{tfewshot}
Article: {inputText}.

Question: What are the relations present in the text? Display it in json format."""
#print(t5query)
inputs = tokenizer(t5query, return_tensors="pt")
outputs = original_model.generate(**inputs, max_new_tokens=100)
output = tokenizer.batch_decode(outputs, skip_special_tokens=True)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{t5query}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{outputText}\n')
print(dash_line)
print(f'MODEL GENERATION - FEW SHOT:\n{output}')

In [ ]:
import datasets
from datasets import Dataset, DatasetDict

train = Dataset.from_pandas(train_df)
valid = Dataset.from_pandas(eval_df)
test = Dataset.from_pandas(test_df)

ds = DatasetDict()

ds['train'] = train
ds['validation'] = valid
ds['test'] = test

print(ds)

In [ ]:
from datasets import concatenate_datasets

# The maximum total input sequence length after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded.
tokenized_inputs = concatenate_datasets([ds["train"], ds["validation"]]).map(lambda x: tokenizer(x["input_text"], truncation=True), batched=True, remove_columns=["input_text", "target_text", "__index_level_0__"])
max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
print(f"Max source length: {max_source_length}")

# The maximum total sequence length for target text after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded."
tokenized_targets = concatenate_datasets([ds["train"], ds["validation"]]).map(lambda x: tokenizer(x["target_text"], truncation=True), batched=True, remove_columns=["input_text", "target_text", "__index_level_0__"])
max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])
print(f"Max target length: {max_target_length}")

In [ ]:
def tokenize_function(example):
    #print(example)
    start_prompt = 'Article: '
    end_prompt = '\n\nQuestion: What are the relations present in the text? Display it in json format.'
    prompt = [start_prompt + sentence + end_prompt for sentence in example["input_text"]]
    #print(prompt)
    example['input_ids'] = tokenizer(prompt, max_length=max_source_length, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["target_text"], max_length=max_target_length, padding="max_length", truncation=True, return_tensors="pt").input_ids
    #print(example)
    return example

# The dataset actually contains 3 diff splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.
tokenized_datasets = ds.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['input_text', 'target_text', '__index_level_0__',])

In [ ]:
def newtokenize_function(example, padding="max_length"):
    #print(example)
    start_prompt = 'Article: '
    #end_prompt = '\n\nQuestion: What are the relations present in the text? Display it in json format.'
    end_prompt = '\n\nQuestion: What are the entities and relations between these entities present in the text? If entities are present then first generate entities and then list relations between entities in lists of lists format.'
    prompt = [start_prompt + sentence + end_prompt for sentence in example["input_text"]]
    #print(prompt)
    model_inputs = tokenizer(prompt, max_length=max_source_length, padding="max_length", truncation=True)
    labels = tokenizer(text=example["target_text"], max_length=max_target_length, padding="max_length", truncation=True)
    #print(example)
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# The dataset actually contains 3 diff splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.
tokenized_datasets = ds.map(newtokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['input_text', 'target_text', '__index_level_0__',])

In [ ]:
#tokenized_datasets = tokenized_datasets.filter(lambda example, index: index % 100 == 0, with_indices=True)
#tokenized_datasets = tokenized_datasets.filter(lambda example, index: index % 100 == 0, with_indices=True)

In [ ]:
print(f"Shapes of the datasets:")
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['validation'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")

print(tokenized_datasets)

In [ ]:
import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

# Metric
metric = evaluate.load("rouge")

# helper function to postprocess text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=original_model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)
#data_collator = DataCollatorForSeq2Seq(tokenizer, model=original_model)

In [ ]:
output_dir = "./feroutputs"
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    num_train_epochs=10,
    weight_decay=0.01,
    predict_with_generate=True,
    overwrite_output_dir=True,
    fp16=False,
    # logging & evaluation strategies
    logging_dir="./logs",
    logging_strategy="steps",
    logging_steps=500,
    save_strategy="epoch",
    save_total_limit=2,
    report_to="none",
)

trainer = Seq2SeqTrainer(
    model=original_model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
trainer.save_model('./feroutputs')

In [10]:
instruct_model = AutoModelForSeq2SeqLM.from_pretrained("./feroutputs/checkpoint-5203")#, torch_dtype=torch.bfloat16)
#tokenizer = AutoTokenizer.from_pretrained("./feroutputs/checkpoint-2028")

In [ ]:
index = 15
dialogue = ds['test'][index]['input_text']
human_baseline_summary = ds['test'][index]['target_text']
#dialogue = "Net Loss: Net loss was $118.9 million , or $1.74 per share with 65 million shares outstanding"
print(dialogue)
index = 3
dialogue2 = ds['test'][index]['input_text']
print(dialogue2)
#prompt = dialogue

ans = "{\"RELATIONS\": [\"KEY:GAAP Net Income Per Share!!TYPE:OUT!!MONEY:$0.38!!LINK:KV\"]}"
sent = ["GAAP Net Income Per Share is $0.38 in second quarter 2023 @@@"]

ans1 = "{\"RELATIONS\": [\"KEY:Non-GAAP net income!!TYPE:OUT!!MONEY:$110.1 MN!!LINK:KV\"]}"
sent1 = ["Non-GAAP net income is $110.1 million in third quarter 2022 @@@"]

ans2 = "{\"RELATIONS\": [\"KEY:Non-GAAP Net Income Per Share!!TYPE:OUT!!MONEY:-$0.13!!LINK:KV\"]}"
sent2 = ["Non-GAAP Net Income Per Share is ($0.13) in first quarter 2023 @@@"]

tfewshot = f"""
Article: {" * ".join(sent)}

Question: What are the entities and relations between these entities present in the text? If entities are present then first generate entities and then list relations between entities in lists of lists format. {ans}

"""
tfewshot += f"""
Article: {" * ".join(sent1)}. 

Question: What are the entities and relations between these entities present in the text? If entities are present then first generate entities and then list relations between entities in lists of lists format. {ans1}

"""
tfewshot += f"""
Article: {" * ".join(sent2)}.

Question: What are the entities and relations between these entities present in the text? If entities are present then first generate entities and then list relations between entities in lists of lists format. {ans2}

"""

prompt = f"""
Article: {dialogue}.

Question: What are the entities and relations between these entities present in the text? If entities are present then first generate entities and then list relations between entities in lists of lists format."""

prompt2 = f"""
Article: {dialogue2}.

Question: What are the entities and relations between these entities present in the text? If entities are present then first generate entities and then list relations between entities in lists of lists format."""


#inputs = tokenizer([prompt], padding="max_length", truncation=True, return_tensors="pt")
inputs = tokenizer(prompt, return_tensors="pt")
outputs = instruct_model.generate(**inputs, max_new_tokens=300, top_p=.9)
#outputs = instruct_model.generate(**inputs)
instruct_output = tokenizer.batch_decode(outputs, skip_special_tokens=True)

#print(prompt)
dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(prompt)
print(prompt2)
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{human_baseline_summary}')
print(dash_line)
print(f'INSTRUCT MODEL:\n{instruct_output}')

In [11]:
line = ""
pmpt = f"""
Article: [ARTICLE].

Question: What are the entities and relations between these entities present in the text? If entities are present then first generate entities and then list relations between entities in lists of lists format."""

def writetofile(inputfile, to_predict, predictions, of, forTrain=False):
    if forTrain:
        return
    for to_pred, preds in zip(to_predict, predictions):
        #print(preds)
        pred = preds
        pred = pred.replace("\n","")
        pred = "{"+pred+"}"
        topred = to_pred.replace("get relations: ","")
        #print(pred, str(type(ast.literal_eval(pred))))
        outstr = ""
        try:
            if(str(type(ast.literal_eval(pred))) == "<class 'dict'>"):
                outstr = (topred + "\t" + pred)
            else:
                outstr = (topred + "\t" + "NO PREDICTION")
        except:
            outstr = (topred + "\t" + "NO PREDICTION")

        #print(outstr)
        #print("\n")
        outstr = inputfile + "\t" + outstr
        of.write(outstr)
        of.write("\n\n")

In [20]:
maxCount = 30

def createDatawithModel(origFile, csym, nlp, erModel, forTrain=True):
    basefile = os.path.basename(origFile)
    inputfile = os.path.splitext(basefile)[0]
    if(not forTrain):
        #print("NOT SUPPORTED WITHOUT TRAINING FLAG AS OF NOW")
        #return
        if not isScanningRqd(origFile):
            print("Format not supported for file {}".format(origFile))
            return False
        
        outdir = "../../Summary/PostRefinedV3"
        outfileDir = outdir+"/"+csym
        if not os.path.exists(outfileDir):
            os.makedirs(outfileDir)
        outfilePath = outfileDir+"/"+inputfile+".txt"
        print(outfilePath)
        outfile = Path(outfilePath)
        if outfile.is_file():
            print(outfilePath + " Already exists")
            return False
        
        print("Creating post refined V3 data file " + str(outfile))
        of = None
        #of = open(outfile, "w", encoding = "utf-8")
        #of.write("filename\tSentence1\tSentence2\n")
        
        header = None
        
        with open(origFile, "r", encoding = "utf-8") as f:
            line = f.readline()
            line = line.strip()
            #line = line.replace("\n", "")
            #print(line)
            to_predict = list()
            while line:
                #print(line)
                if("ED***" in line):
                    break
                elif("CS***" in line or "TBLST***" in line or "TBLET***" in line):
                    if(len(to_predict) > 0):
                        print(to_predict)
                        print("\n\n")
                        #predictions = erModel.predict(to_predict)
                        #print(predictions)
                        #print("\n\n")
                        #writetofile(inputfile, to_predict, predictions, of, forTrain=False)
                    to_predict = list()
                    line = line.replace("\n","")
                    #print(line + " #### " + "NOT PROCCESSED")
                    #print("\n")
                    #sentences = getSentences(None, nlp, line)
                    #for l in sentences:
                    #    if("CS***" in line):
                    #        if("CS***" in l):
                    #            of.write(inputfile + "\t" + l + "\t" + "{\"ENTITIES\": [null], \"RELATIONS\": [\"\"]}")
                    #        else:
                    #            of.write(inputfile + "\t" + "CS*** "+l + "\t" + "{\"ENTITIES\": [null], \"RELATIONS\": [\"\"]}")
                    #    else:
                    #        of.write(inputfile + "\t" + l + "\t" + "{\"ENTITIES\": [null], \"RELATIONS\": [\"\"]}")
                    #    of.write("\n\n")
                    line = f.readline()
                    continue
                #elif("PG***" in line or "NOPAD***" in line or ("***" not in line)):
                else:
                    tag = line.split(" ")[0].strip()
                    #print(tag)
                    #print(sent)
                    if(tag in stag):
                        section = line.replace(tag, "").strip()
                        header = section
                        #print(header)
                        nl = pmpt.replace("[ARTICLE]", line)
                        to_predict = to_predict + ["get relations: "+nl]
                    elif(tag in gtag):
                        ssection = line.replace(tag, "").strip()
                        header = ssection
                        #print(header)
                        nl = pmpt.replace("[ARTICLE]", line)
                        to_predict = to_predict + ["get relations: "+nl]
                    else:
                        sentences = getSentences(None, nlp, line)
                        #to_predict = list()
                        for l in sentences:
                            #print(l)
                            pline, isProcess = preProcessSent(l)
                            if not isProcess:
                                continue
                            if(header):
                                if("***" in tag):
                                    nl = l.replace(tag, "")
                                    nl = tag + " " + header + " -%%%- " + nl
                                else:
                                    nl = header + " -%%%- " + l
                            else:
                                nl = l
                            nl = pmpt.replace("[ARTICLE]", nl)
                            to_predict = to_predict + [nl]
                            if(len(to_predict) >= maxCount):
                                print(to_predict)
                                print("\n\n")
                                #predictions = erModel.predict(to_predict)
                                #print(predictions)
                                #print("\n\n")
                                #writetofile(inputfile, to_predict, predictions, of, forTrain=False)
                                to_predict = list()
                line = f.readline()
            
            if(len(to_predict) > 0):
                print(to_predict)
                print("\n\n")
                #predictions = erModel.predict(to_predict)
                #print(predictions)
                #print("\n\n")
                #writetofile(inputfile, to_predict, predictions, of, forTrain=False)
                to_predict = list()
        if(of):
            of.close()
            
    return True

In [21]:
entPath = "../../Summary/entities/"
rPath = "../../Summary/Refined/"
files = glob.glob(entPath+"/*-ENTITIES.json")   
if(len(files) > 0):
    for file in (files):
        basefile = os.path.basename(file)
        filename = os.path.splitext(basefile)[0]
        csym = filename.split("-")[0]
        cPath = rPath + "/" + csym
        #print(cPath)
        cfiles = glob.glob(cPath+"/*.txt")
        #print(cfiles)
        if(len(cfiles) > 0):
            for cf in cfiles:
                if isScanningRqd(cf):
                    print(cf)
                    created = createDatawithModel(cf, csym, nlp, instruct_model, forTrain=False)
                    if not created:
                        continue
                    break
        if created:
            break

../../Summary/Refined//APPN\APPN_2022-08-04_EP_YH.txt
../../Summary/PostRefinedV3/APPN/APPN_2022-08-04_EP_YH.txt
Creating post refined V3 data file ..\..\Summary\PostRefinedV3\APPN\APPN_2022-08-04_EP_YH.txt
['\nArticle: GAAP Net Loss Per Share is $(0.68) in second quarter 2022 @@@.\n\nQuestion: What are the entities and relations between these entities present in the text? If entities are present then first generate entities and then list relations between entities in lists of lists format.', '\nArticle: Cash And Cash Equivalents is $76185 T in second quarter 2022 @@@.\n\nQuestion: What are the entities and relations between these entities present in the text? If entities are present then first generate entities and then list relations between entities in lists of lists format.', '\nArticle: GAAP Gross Profit is $76770 T in second quarter 2022 @@@.\n\nQuestion: What are the entities and relations between these entities present in the text? If entities are present then first generate en

['get relations: \nArticle: GQ*** Third Quarter 2022 Guidance:.\n.\n\nQuestion: What are the entities and relations between these entities present in the text? If entities are present then first generate entities and then list relations between entities in lists of lists format.', '\nArticle: PG*** Third Quarter 2022 Guidance:. -%%%-  Cloud subscription revenue is expected to be between $60.8 million and $61.3 million, representing year-over-year growth of 30% to 31%..\n\nQuestion: What are the entities and relations between these entities present in the text? If entities are present then first generate entities and then list relations between entities in lists of lists format.', '\nArticle: PG*** Third Quarter 2022 Guidance:. -%%%-  Total revenue is expected to be between $115.0 million and $117.0 million, representing a year-over-year increase of 24% to 27%..\n\nQuestion: What are the entities and relations between these entities present in the text? If entities are present then firs